# Gremlin Workshop - Section 08: Participant Exercises

Here are the solutions to the exercises in Section 07.

## Graph Model

As we have in previous notebooks, we've included here a copy of the graph data model for this dataset as reference to aid in writing your queries.

<img src="https://ee-assets-prod-us-east-1.s3.amazonaws.com/modules/f3f89ef4607743429fb01ae23d983197/v1/workshop/images/datamodel-updated.png"/>


### 08.01 - Simple look-up by property

Display all the properties (using _valueMap()_) of artist with name Lucy Liu.  NOTE: You'll want to end this query with a `limit(1)`, as there are many Lucy Lius, but we're looking primarily for the first one in our dataset.



In [ ]:
%%gremlin --store-to results_08_01

g.V().hasLabel('Artist').has('name','Lucy Liu').valueMap().limit(1)

In [ ]:
!pip install ipython_unittest
%load_ext ipython_unittest

In [ ]:
%%unittest

assert results_08_01 == [{'birthYear': [1968], 'name': ['Lucy Liu']}]

### 08.02 - Single hop traversal

Find out all the movies which Denzel Washington acted in. (Order the movies by id before limiting to 10.)



In [ ]:
%%gremlin --store-to results_08_02

g.V().hasLabel('Artist').has('name', 'Denzel Washington').in('actor').
                 hasLabel('movie').order().by(id).limit(10).values('title')

In [ ]:
%%unittest

assert results_08_02 == ['Cry Freedom', 
                         'For Queen & Country', 
                         'Glory', 
                         'The Mighty Quinn', 
                         'Heart Condition', 
                         "Mo' Better Blues", 
                         'Mississippi Masala', 
                         'Ricochet', 
                         'Malcolm X', 
                         'The Pelican Brief']

### 08.03 - Two hop traversal

Find 5 unique actors or actresses which were paired with Julia Roberts. (Use _dedup()_ to deduplicate the actors, and then order by id before limiting to 5.)


In [ ]:
%%gremlin --store-to results_08_03

g.V().hasLabel('Artist').has('name', 'Julia Roberts').
                 in('actress').
                 out('actor','actress').
                 dedup().order().by(id).limit(5).
                 values('name')

In [ ]:
%%unittest

assert results_08_03 == ['Sophia Loren', 
                         'Marcello Mastroianni', 
                         'Brad Pitt', 
                         'Woody Allen', 
                         'Jennifer Aniston']

### 08.04 - Using traversal filters

Find the Person who lives in Belfast and follows Gillian Anderson. (See [example 3 of the _where()_ step](http://tinkerpop.apache.org/docs/current/reference/#where-step) for how to use a traversal while filtering.)


In [ ]:
%%gremlin --store-to results_08_04

g.V().hasLabel('Artist').has('name', 'Gillian Anderson').
                 in('follows').
                 where(out('isLocatedIn').values('name').is(eq('Belfast'))).
                 values('firstName', 'lastName').fold()

In [ ]:
%%unittest

assert results_08_04 == [['David', 'Bell']]

### 08.05 - Using filter on edges & range queries

List persons who have given The Presitige a rating of less than 3. (Use _inE()_ or _outE()_ to position the traverser on an edge. Use _project()_ to construct your results.)


In [ ]:
%%gremlin --store-to results_08_05

g.V().hasLabel('movie').has('title', 'The Prestige').
                 inE('rated').has('rating',lt(3)).
                 otherV().
                 project('firstName', 'lastName').
                 by('firstName').
                 by('lastName')

In [ ]:
%%unittest

assert results_08_05 == [{'firstName': 'Amber', 'lastName': 'Azami'}, 
                            {'firstName': 'Paulo', 'lastName': 'Silva'},
                            {'firstName': 'Jharana','lastName': 'Gurung'}]

### 08.06 - Aggregating and Counting Results

Find all Artists that worked with John Cleese and the number of times they appeared with him. Order the results by number of appearances and return the first five. (Hint: Use `groupCount()` )

In [ ]:
%%gremlin --store-to results_08_06

g.V().hasLabel('Artist').has('name','John Cleese').as('john').
         in('actor','actress').out('actor','actress').where(neq('john')).
         groupCount().by('name').unfold().order().by(values,Order.desc).limit(5)

In [ ]:
%%unittest

assert results_08_06 == [{'Michael Palin': 6}, 
                         {'Graham Chapman': 5}, 
                         {'Terry Gilliam': 5}, 
                         {'Eric Idle': 4}, 
                         {'Terry Jones': 3}]